In [1]:
import os
import time
import shutil
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils as utils
import torch.nn.init as init
from torchvision import datasets

In [2]:
data_type = 'mem2d'
model_num = 1
time_len =  20  ## ms
point_len = int(time_len*10)
fc_len = int(point_len/32)+1

In [3]:
class Model(nn.Module):
    def __init__(self, loss):
        super(Model, self).__init__()
        prob = 0.8
        input_c = 10
        channel = 16
        self.conv1 = nn.Sequential(
            nn.Conv2d(input_c, channel, kernel_size=(1,5), stride=(1,2), padding=(0,2)),
#             nn.Dropout(p=prob),
            nn.LeakyReLU(negative_slope=0.1),
            nn.BatchNorm2d(channel))
        self.conv2 = nn.Sequential(
            nn.Conv2d(channel, channel*2, kernel_size=(1,5), stride=(1,2), padding=(0,2)),
#             nn.Dropout(p=prob),
            nn.LeakyReLU(negative_slope=0.1),
            nn.BatchNorm2d(channel*2))
        self.conv3 = nn.Sequential(
            nn.Conv2d(channel*2, channel*2, kernel_size=(1,5), stride=(1,2), padding=(0,2)),
#             nn.Dropout(p=prob),
            nn.LeakyReLU(negative_slope=0.1),
            nn.BatchNorm2d(channel*2))
        self.conv4 = nn.Sequential(
            nn.Conv2d(channel*2, channel*4, kernel_size=(1,5), stride=(1,2), padding=(0,2)),
#             nn.Dropout(p=prob),
            nn.LeakyReLU(negative_slope=0.1),
            nn.BatchNorm2d(channel*4))
        self.conv5 = nn.Sequential(
            nn.Conv2d(channel*4, channel*4, kernel_size=(1,5), stride=(1,2), padding=(0,2)),
#             nn.Dropout(p=prob),
            nn.LeakyReLU(negative_slope=0.1),
            nn.BatchNorm2d(channel*4))
        self.conv6 = nn.Sequential(
            nn.Conv2d(channel*4, channel*8, kernel_size=(1,5), stride=(1,2), padding=(0,2)),
#             nn.Dropout(p=prob),
            nn.LeakyReLU(negative_slope=0.1),
            nn.BatchNorm2d(channel*8))
        self.conv7 = nn.Sequential(
            nn.Conv2d(channel*8, channel*8, kernel_size=(1,5), stride=(1,2), padding=(0,2)),
#             nn.Dropout(p=prob),
            nn.LeakyReLU(negative_slope=0.1),
            nn.BatchNorm2d(channel*8))
        self.fc1 = nn.Sequential(
            nn.Linear(channel*4*fc_len, 1024),
#             nn.Dropout(p=prob),
            nn.LeakyReLU(negative_slope=0.1),
            nn.BatchNorm1d(1024))
        self.fc2 = nn.Sequential(
            nn.Linear(1024, 5),
            nn.Softmax(dim=1))
        self.loss = loss
        
    def forward(self, data, target):
        x = self.conv1(data)
#         print('1',x.size())
        x = self.conv2(x)
#         print('2',x.size())
        x = self.conv3(x)
#         print('3',x.size())
        x = self.conv4(x)
#         print('4',x.size())
        x = self.conv5(x)
#         print('5',x.size())
#         x = self.conv6(x)
#         print('6',x.size())
#         x = self.conv7(x)
#         print('7',x.size())
        x = x.view(x.size()[0],-1)
#         print(x.size())
        x = self.fc1(x)
#         print('8',x.size())
        h = self.fc2(x)
#         print('9',h.size())
        
        l = self.loss(h, target)
        return l, h, target

In [4]:
def train(model, trainX, trainY, batch, device, optimizer, train_loss, train_acc):
    model.train()
    dloss = 0
    dacc = 0
    
    rand = torch.randperm(trainX.size()[0])
    trainX = trainX[rand]
    trainY = trainY[rand]
    
    for i in range(batch[0]):
        optimizer.zero_grad()
        loss, output, target = model(trainX[i*batch[1]:(i+1)*batch[1]], trainY[i*batch[1]:(i+1)*batch[1]])
        loss = loss.sum()
        loss.backward()
        optimizer.step()
        
        _, output = torch.max(output, 1)
        _, target = torch.max(target, 1)

        dloss += loss.cpu().item()
        dacc += (output==target).sum().item()
        
    train_loss.append(dloss/batch[0])
    train_acc.append(dacc/(batch[0]*batch[1]))
    return train_loss, train_acc

def test(model, testX, testY, device, test_loss, test_acc):
    model.eval()
    loss, output, target = model(testX, testY)
    loss = loss.sum()
    
    _, output = torch.max(output, 1)
    _, target = torch.max(target, 1)
    
    test_loss.append(loss.cpu().item())
    test_acc.append((output==target).sum().item()/len(testX))
    return test_loss, test_acc

In [5]:
if __name__=='__main__':
    print('[Training]%s_%.1f_model%d'%(data_type, time_len, model_num))
    torch.manual_seed(37)
    torch.cuda.manual_seed_all(37)
    torch.backends.cudnn.deterministic = True
    
    trainX = np.load('npy_data/%s/%s_%.1fms_trainX.npy'%(data_type[:-2], data_type[:-2], time_len)).reshape(-1, 10, 1, point_len)
    trainY = np.load('npy_data/%s/%s_%.1fms_trainY.npy'%(data_type[:-2], data_type[:-2], time_len))
    testX = np.load('npy_data/%s/%s_%.1fms_testX.npy'%(data_type[:-2], data_type[:-2], time_len)).reshape(-1, 10, 1, point_len)
    testY = np.load('npy_data/%s/%s_%.1fms_testY.npy'%(data_type[:-2], data_type[:-2], time_len))

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    batch_num = 1000
    batch = [int(len(trainX)/batch_num), batch_num]
    
    trainX = torch.Tensor(trainX).to(device)
#     _, trainY = torch.max(torch.LongTensor(trainY).to(device), 1)
    trainY = torch.Tensor(trainY).to(device)
    testX = torch.Tensor(testX).to(device)
    testY = torch.Tensor(testY).to(device)
#     _, testY = torch.max(torch.LongTensor(testY).to(device), 1)
    print('mem data shape  -  %.1f ms'%time_len)
    print('train set :', np.shape(trainX) , np.shape(trainY))
    print('test set :', np.shape(testX) ,np.shape(testY))
    
    

[Training]mem2d_20.0_model1
mem data shape  -  20.0 ms
train set : torch.Size([5000, 10, 1, 200]) torch.Size([5000, 5])
test set : torch.Size([1500, 10, 1, 200]) torch.Size([1500, 5])


In [6]:
    learning_rate = 0.00005
#     loss_func = nn.CrossEntropyLoss
    loss_func=nn.BCELoss()
    model = nn.DataParallel(Model(loss_func)).to(device)
#     model = Model(loss_func).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=batch_num, eta_min = 3e-6)

#     model.load_state_dict(torch.load('ckpt/model%d_mem/%.1f_ckpt_2000.pt'%(model_num, time_len)))
    a = time.time()
    train_loss = []
    train_acc = []
    test_loss = []
    test_acc = []
    for epoch in range(1001):
        train_loss, train_acc = train(model, trainX, trainY, batch, device, optimizer, train_loss, train_acc)
        test_loss, test_acc = test(model, testX, testY, device, test_loss, test_acc)
        scheduler.step()
        
        if epoch%10==0: 
            print('epoch %d - train loss : %.7f  /  test loss : %.7f'%(epoch, train_loss[-1], test_loss[-1]))
            print('           train acc : %.7f  /  test acc : %.7f'%(train_acc[-1], test_acc[-1]))
#         if epoch%50==0:
#             print('@@@@@@@ save model : epoch %d'% epoch)
#             torch.save(model.state_dict(),'ckpt/model%d_%s/%.1f_ckpt_%d.pt'%(model_num, data_type, time_len, epoch))
#             np.savetxt('result/model%d_%s/%.1f_loss_tr.txt'%(model_num, data_type, time_len), train_loss)
#             np.savetxt('result/model%d_%s/%.1f_loss_te.txt'%(model_num, data_type, time_len), test_loss)
#             np.savetxt('result/model%d_%s/%.1f_acc_tr.txt'%(model_num, data_type, time_len), train_acc)
#             np.savetxt('result/model%d_%s/%.1f_acc_te.txt'%(model_num, data_type, time_len), test_acc)
    print("training complete! - calculation time :", time.time()-a, '  seconds')

/opt/anaconda3/envs/khj/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


epoch 0 - train loss : 0.9879306  /  test loss : 1.0008135
           train acc : 0.2772000  /  test acc : 0.2000000
epoch 10 - train loss : 0.6424380  /  test loss : 1.5431302
           train acc : 0.6528000  /  test acc : 0.2386667
epoch 20 - train loss : 0.3989997  /  test loss : 0.4527657
           train acc : 0.7912000  /  test acc : 0.7446667
epoch 30 - train loss : 0.2675799  /  test loss : 0.3335437
           train acc : 0.8786000  /  test acc : 0.8360000
epoch 40 - train loss : 0.1755678  /  test loss : 0.2596631
           train acc : 0.9282000  /  test acc : 0.8713333
epoch 50 - train loss : 0.1136863  /  test loss : 0.2346817
           train acc : 0.9576000  /  test acc : 0.8793333
epoch 60 - train loss : 0.0743654  /  test loss : 0.2320848
           train acc : 0.9762000  /  test acc : 0.8780000
epoch 70 - train loss : 0.0457981  /  test loss : 0.2391125
           train acc : 0.9920000  /  test acc : 0.8760000
epoch 80 - train loss : 0.0279647  /  test loss : 0.25078

KeyboardInterrupt: 